In [1]:
# Import Packages
import pandas as pd
import random

In [2]:
# Columns
from_city = ["A", "A", "A", "B", "B", "B", "C", "C", "C", "D", "D", "D"]
to_city = ["B", "C", "D", "A", "C", "D", "A", "B", "D", "A", "B", "C"]
distance = [5, 2, 9, 5, 11, 3, 2, 11, 4, 9, 3, 4]
attractive_to = [33, 45, 76, 68, 45, 76, 68, 33, 76, 68, 33, 45]
presence = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]

# Create DF
df = pd.DataFrame({"From": from_city, "To": to_city, "Lag": distance, "W": attractive_to, "Starting Presence": presence})

df

,From,To,Lag,W,Starting Presence
0,A,B,5,33,0
1,A,C,2,45,0
2,A,D,9,76,0
3,B,A,5,68,1
4,B,C,11,45,1
5,B,D,3,76,1
6,C,A,2,68,0
7,C,B,11,33,0
8,C,D,4,76,0
9,D,A,9,68,0


In [3]:
# Weight / Distance
df["Wi/Dij"] = df["W"] / df["Lag"]

# Get Sum
wd_sum = df["Wi/Dij"].sum()

# Calculate Huff
df["Huff"] = df["Wi/Dij"] / wd_sum

# Add Transition Count
df["Transition Count"] = 0

# Set End Presence to 0
df["End Presence"] = 0

df

,From,To,Lag,W,Starting Presence,Wi/Dij,Huff,Transition Count,End Presence
0,A,B,5,33,0,6.600000,0.039670,0,0
1,A,C,2,45,0,22.500000,0.135237,0,0
2,A,D,9,76,0,8.444444,0.050756,0,0
3,B,A,5,68,1,13.600000,0.081743,0,0
4,B,C,11,45,1,4.090909,0.024589,0,0
5,B,D,3,76,1,25.333333,0.152267,0,0
6,C,A,2,68,0,34.000000,0.204359,0,0
7,C,B,11,33,0,3.000000,0.018032,0,0
8,C,D,4,76,0,19.000000,0.114200,0,0
9,D,A,9,68,0,7.555556,0.045413,0,0


In [4]:
def single_simulation():
    # Loop through Rows & Simulate Transfer
    for index, row in df.iterrows():
        if row["Starting Presence"] == 1:
            # Generate Random Number
            n = random.random()
            
            # Check if n < Huff --- Seems like this might not be the best/correct way to do this
            if n < row["Huff"]:
                df.loc[index, "End Presence"] = 1
                df.loc[index, "Transition Count"] += 1
                
    # Get List of Cities with End Presence
    end_presence = list(df.loc[df["End Presence"] == 1]["To"])

    # Reset Starting Presence to 0
    df["Starting Presence"] = 0

    # Set New Starting Presence
    df.loc[df["From"].isin(end_presence), ["Starting Presence"]] = 1

    # If All Vals are 0, reset to initial settings
    if (df["Starting Presence"] == 0).all() == True:
        df["Starting Presence"] = presence

In [5]:
for i in range(1000):
    single_simulation()
    
df

,From,To,Lag,W,Starting Presence,Wi/Dij,Huff,Transition Count,End Presence
0,A,B,5,33,1,6.600000,0.039670,41,1
1,A,C,2,45,1,22.500000,0.135237,133,1
2,A,D,9,76,1,8.444444,0.050756,52,1
3,B,A,5,68,1,13.600000,0.081743,72,1
4,B,C,11,45,1,4.090909,0.024589,28,1
5,B,D,3,76,1,25.333333,0.152267,162,1
6,C,A,2,68,1,34.000000,0.204359,198,1
7,C,B,11,33,1,3.000000,0.018032,22,1
8,C,D,4,76,1,19.000000,0.114200,114,1
9,D,A,9,68,1,7.555556,0.045413,38,1


In [6]:
# Total Incoming Bugs per City - one way we can prioritize transmission hazard
df.groupby("To").agg(["sum"])["Transition Count"]

,sum
To,
A,308
B,125
C,233
D,328


In [8]:
# Total Outgoing Bugs per City - one way we can prioritize transmission hazard
df.groupby("From").agg(["sum"])["Transition Count"]

,sum
From,
A,232
B,273
C,355
D,168


In [9]:
# Total Percentage Risk to City - one way we can prioritize transmission hazard
# High values show centrally located cities with high weights/attractiveness; opposite for low values
df.groupby("To").agg(["sum"])["Huff"]

,sum
To,
A,0.331515
B,0.123817
C,0.227445
D,0.317223
